# Stacks and Queues

## Agenda

1. Stacks
     - ... for delimiter pairing
     - ... for postfix expression evaluation
     - ... for tracking execution and *backtracking*
2. Queues
    - ... for tracking execution and *backtracking*
    - ... for fair scheduling (aka "round-robin" scheduling)
    - ... for apportioning work
3. Run-time analysis

## Overview

While the list data structure is incredibly useful, both implementations we explored (array-backed and linked) have operations that run in $O(N)$ time, which make them non-ideal for use with large, growing collections of data.

By further restricting the list API, however — in particular, by *isolating points of access to either the front or end of the data set* — we can create data structures whose operations are all $O(1)$, and remain very useful in their own right.

## 1. Stacks

Stacks are linear data structures which only permit access to one "end" of the data collection. We can only append ("push") items onto the tail end (a.k.a. the "top") of a stack, and only the most recently added item can be removed ("popped"). The last item to be pushed onto a stack is therefore the first one to be popped off, which is why we refer to stacks as last-in, first out (LIFO) structures.

In [1]:
# array-backed implementation

class Stack: #has to be constant time all around
    def __init__(self):
        self.data = [] #array backed list as implementation container
        
    def push(self, val): #add val at end of arraylist
        self.data.append(val)

    def pop(self): #delete last value at the end of arraylist
        assert not self.empty()
        ret = self.data[-1]
        del self.data[-1]
        return ret #constant time
    
    def peek(self): #view last slot of stack
        assert not self.empty()
        return self.data[-1]

    def empty(self): #returns true if the stack is empty
        return len(self.data) == 0

    def __bool__(self): #evaluates to not empty
        return not self.empty()
    #used when doing if self

In [2]:
s = Stack()
for x in range(10):
    s.push(x)

In [3]:
while s: #uses bool method, executes when true
    print(s.pop())

9
8
7
6
5
4
3
2
1
0


In [4]:
# linked implementation

class Stack:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next  = next
        #singly linked stacked
    
    def __init__(self):
        self.top = None #top element of stack

    def push(self, val):
        self.top = Stack.Node(val, next=self.top)
        
    def pop(self):
        assert not self.empty()
        ret = self.top.val
        self.top = self.top.next
        return ret

    def peek(self):
        assert not self.empty()
        return self.top.val
        
    def empty(self):
        return self.top is None
    
    def __bool__(self):
        return not self.empty()

In [5]:
s = Stack()
for x in range(10):
    s.push(x)

In [6]:
while s:
    print(s.pop())

9
8
7
6
5
4
3
2
1
0


### ... for delimiter pairing

Stacks are used by parsers to decide if delimited expressions are well-formed.

e.g., `'(1 + 2 * (3 - (4 / 2) + 5) - (6 + 1))'`

In [24]:
def check_parens(expr):
    s = Stack()
    for c in expr:
        if c == '(':
            s.push(c)
        elif c == ")":
            try:
                assert not s.empty()
                assert s.pop() == '('
            except:
                return False
    return s.empty()

In [25]:
check_parens('()')

True

In [26]:
check_parens('((()))')

True

In [27]:
check_parens('()(()()(()))')

True

In [28]:
check_parens('(')

False

In [29]:
check_parens('())')

False

In [30]:
check_parens('(1 + 2 * (3 - (4 / 2) + 5) - (6 + 1))')

True

### ... for postfix expression (aka "reverse polish notation") evaluation

Stacks are used for the evaluation of postfix arithmetic expressions (which can be easily converted back and forth between the more common infix expressions). 

e.g., `'(1 + 2) * 5'` $\rightarrow$ `'1 2 + 5 *'`

In [33]:
def eval_postfix(expr):
    s = Stack()
    toks = expr.split()
    for t in toks:
        if t == '+':
            s.push(s.pop() + s.pop())
        elif t == '*':
            s.push(s.pop() * s.pop())
        else:
            s.push(int(t))
    return s.pop()

In [34]:
eval_postfix('1 2 + 5 *')

15

In [35]:
eval_postfix('1 2 5 * +')

11

In [36]:
# ((1 + 2) * (3 + 2)) * 10
eval_postfix('1 2 + 3 2 + * 10 *')

150

### ... for tracking execution and backtracking

In [37]:
maze_str = """######   
              I    #   
              # ## #   
              # ####   
              #    O   
              ######"""

def parse_maze(maze_str):
    '''Parses a string representing a maze into a 2D array.'''
    grid = []
    for line in maze_str.split('\n'):
        grid.append(['# IO'.index(c) for c in line.strip()])
    return grid

def print_maze(grid):
    '''Takes a 2D array maze representation and pretty-prints it.
       The contents of the 2D maze are in the range 0-5, which are interpreted as:
    
        0: a wall
        1: an unvisited (i.e., not previously traversed) path
        2: the maze entrance
        3: the maze exit
        4: a discovered but unvisited path
        5: a visited path
    '''
    for r in grid:
        print(''.join('# IO!+'[c] for c in r))

In [38]:
parse_maze(maze_str)

[[0, 0, 0, 0, 0, 0],
 [2, 1, 1, 1, 1, 0],
 [0, 1, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 3],
 [0, 0, 0, 0, 0, 0]]

In [39]:
print_maze(parse_maze(maze_str))

######
I    #
# ## #
# ####
#    O
######


In [40]:
maze = parse_maze(maze_str)
maze[1][0] = maze[1][1] = 5
maze[1][2] = maze[2][1] = 4
print_maze(maze)

######
++!  #
#!## #
# ####
#    O
######


In [41]:
class Move:
    '''Represents a move in the maze between orthogonally adjacent locations
      `frm` and `to`, which are both (row,col) tuples.'''
    def __init__(self, frm, to):
        self.frm = frm
        self.to  = to
        
    def __repr__(self):
        return '({},{}) -> ({},{})'.format(self.frm[0], self.frm[1],
                                           self.to[0],  self.to[1])

def moves(maze, loc):
    '''Returns all possible moves within a maze from the provide location.'''
    moves = [Move(loc, (loc[0]+d[0], loc[1]+d[1]))
            for d in ((-1, 0), (1, 0), (0, -1), (0, 1))
            if loc[0]+d[0] in range(len(maze)) and 
               loc[1]+d[1] in range(len(maze[0])) and
               maze[loc[0]+d[0]][loc[1]+d[1]] in (1, 2, 3)]
    return moves

In [42]:
maze = parse_maze(maze_str)
print_maze(maze)

######
I    #
# ## #
# ####
#    O
######


In [44]:
moves(maze, (1, 0)) #can only going right, so its (1, 1)

[(1,0) -> (1,1)]

In [45]:
moves(maze, (1, 1))

[(1,1) -> (2,1), (1,1) -> (1,0), (1,1) -> (1,2)]

In [46]:
maze[1][0] = 5
moves(maze, (1, 1))

[(1,1) -> (2,1), (1,1) -> (1,2)]

In [55]:
from time import sleep
from IPython.display import clear_output

def mark(maze, loc):
    '''Marks a loc in the maze as having been discovered'''
    if maze[loc[0]][loc[1]] != 3:
        maze[loc[0]][loc[1]] = 4

def visit(maze, loc):
    '''Marks a loc in the maze as having been visited'''
    maze[loc[0]][loc[1]] = 5    
    
def display(maze):
    '''Prints out the maze after clearing the cell -- useful for animation.'''
    clear_output(True)
    print_maze(maze)
    sleep(0.7)

In [56]:
def solve_maze(maze, entry):
    '''Searches for the exit in a maze starting from the given entry point.
    
       The algorithm works as follows:
       
       1. Visit the entry point and save all possible moves from that location.
       2. Remove and consider one of the saved moves. If it is the exit, we are done, 
          otherwise visit the destination and save all possible moves from there.
       3. If we run out of saved moves, we can't find an exit.
       
       When we save a move, we also mark it as "discovered" in the maze.
       
       The data structure used to save moves plays a critical role in how maze
       exploration proceeds! 
    '''
    for m in moves(maze, entry):
        save_move(m)
    visit(maze, entry)
    while not out_of_moves():
        move = next_move()
        if maze[move.to[0]][move.to[1]] == 3:
            break
        display(maze)
        visit(maze, move.to)
        for m in moves(maze, move.to):
            mark(maze, m.to)
            save_move(m)
    display(maze)

In [57]:
move_stack = Stack()

def save_move(move):
    move_stack.push(move)

def next_move():
    return move_stack.pop()

def out_of_moves():
    return move_stack.empty()

In [53]:
maze_str = """######   
              I    #   
              # ## #   
              # ####   
              #    O   
              ######"""
solve_maze(parse_maze(maze_str), (1, 0))

######
+++++#
#+##+#
#+####
#++++O
######


In [54]:
maze_str = """#################
              I #       #     #
              # ##### # # # # #
              #     # # # # # #
              # ### ### # # ###
              #   #       #   O
              #################"""

solve_maze(parse_maze(maze_str), (1, 0))

#################
++#       #+++++#
#+##### # #+#+#+#
#+++++# # #+#+#+#
#!###+###!#+#+###
#   #+++++++#+++O
#################


In [58]:
maze_str = """#################
              I               #
              # # # # # # # # #
              # # # # # # # # #
              # ###############
              #               O
              #################"""

solve_maze(parse_maze(maze_str), (1, 0))

#################
++++++++++++++++#
#+#+#+#+#+#+#+#+#
#+#+#+#+#+#+#+#+#
#+###############
#+++++++++++++++O
#################


Intuitively, because the stack is a last-in, first-out data structure, it keeps following moves down the most recently discovered path until it either reaches the exit or reaches a dead end. It then picks up from the previously discovered path. We call this type of exploration *depth-first traversal*.

## 2. Queues

Queues are linear data structures wherein we are only permitted to append ("enqueue") items onto the rear, and remove ("dequeue") items from the front. The oldest item still in a queue is therefore the next one to be dequeued, which is why we refer to a queue as a first-in, first-out (FIFO) structure. It is helpful to think of a queue as being the model for a line at a typical supermarket checkout aisle (first customer in, first customer to be checked out).

In [59]:
# array-backed implementation

class Queue:
    def __init__(self):
        self.data = []

    def enqueue(self, val):
        self.data.append(val)
        
    def dequeue(self):
        assert not self.empty()
        ret = self.data[0]
        del self.data[0] #O(N)
        return ret
    
    def empty(self):
        return len(self.data) == 0
        
    def __bool__(self):
        return not self.empty()

In [60]:
q = Queue()
for x in range(10):
    q.enqueue(x)

In [61]:
while q:
    print(q.dequeue())

0
1
2
3
4
5
6
7
8
9


In [62]:
# circular array-backed implementation (partial)

class Queue:
    def __init__(self, size):
        self.data = [None] * size
        self.head = self.tail = -1

    def enqueue(self, val):
        if self.head == -1:
            self.data[0] = val
            self.head = self.tail = 0
        else: 
            self.tail = (self.tail + 1) % len(self.data)
            self.data[self.tail] = val 
        
    def dequeue(self):
        ret = self.data[self.head]
        self.data[self.head] = None
        self.head = (self.head + 1) % len(self.data)
        return ret

In [63]:
q = Queue(10)
for x in range(6):
    q.enqueue(x)

In [64]:
q.data

[0, 1, 2, 3, 4, 5, None, None, None, None]

In [65]:
for x in range(5):
    print(q.dequeue())

0
1
2
3
4


In [66]:
q.data

[None, None, None, None, None, 5, None, None, None, None]

In [67]:
for x in range(6, 12):
    q.enqueue(x)

In [68]:
q.data

[10, 11, None, None, None, 5, 6, 7, 8, 9]

In [69]:
q.head, q.tail

(5, 1)

In [72]:
# linked implementation

class Queue:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next  = next
    
    def __init__(self):
        self.head = self.tail = None

    def enqueue(self, val):
        if not self.empty():
            self.tail.next = self.tail = Queue.Node(val)
        else:
            self.head = self.tail = Queue.Node(val)
            
    def dequeue(self):
        assert not self.empty()
        ret = self.head.val
        self.head = self.head.next
        if not self.head: 
            self.tail = None
        return ret
    
    def empty(self):
        return self.head is None

    def __bool__(self):
        return not self.empty()

In [73]:
q = Queue()
for x in range(10):
    q.enqueue(x)

In [74]:
while q:
    print(q.dequeue())

0
1
2
3
4
5
6
7
8
9


### ... for tracking execution and backtracking

In [77]:
move_queue = Queue()

def save_move(move):
    move_queue.enqueue(move)

def next_move():
    return move_queue.dequeue()

def out_of_moves():
    return move_queue.empty()

In [78]:
maze_str = """######   
              I    #   
              # ## #   
              # ####   
              #    O   
              ######"""

solve_maze(parse_maze(maze_str), (1, 0))

######
+++++#
#+##+#
#+####
#++++O
######


In [79]:
maze_str = """#################
              I #       #     #
              # ##### # # # # #
              #     # # # # # #
              # ### ### # # ###
              #   #       #   O
              #################"""

solve_maze(parse_maze(maze_str), (1, 0))

#################
++#+++++++#+++++#
#+#####+#+#+#+#+#
#+++++#+#+#+#+#+#
#+###+###+#+#+###
#+++#+++++++#+++O
#################


In [80]:
maze_str = """#################
              I               #
              # # # # # # # # #
              # # # # # # # # #
              # ###############
              #               O
              #################"""

solve_maze(parse_maze(maze_str), (1, 0))

#################
++++++++++++++++#
#+#+#+#+#+#+#+#+#
#+#+#+#+#+#+#+#+#
#+###############
#+++++++++++++++O
#################


Intuitively, because the queue is a first-in, first-out -- i.e., *fair* -- data structure, it keeps rotating through all the paths which haven't yet dead-ended, making just one move further down each time. We call this type of exploration *breadth-first traversal*.

Are there types of mazes which might be more suitably tackled using one approach over the other (i.e., depth-first vs. breadth-first)? 

### ... for fair scheduling (aka "round-robin" scheduling)

Queues are often used to either carry out or simulate the fair allocation of resources. Here we implement a "round-robin" scheduler for permitting different tasks to run for small, fixed periods of time until they complete:

In [81]:
from random import randint
from time import sleep

task_queue = Queue()
for i in range(3):
    task_queue.enqueue(('Job {}'.format(i), randint(3, 6)))

In [82]:
n = task_queue.head
while n:
    print(n.val)
    n = n.next

('Job 0', 3)
('Job 1', 6)
('Job 2', 5)


In [83]:
while not task_queue.empty():
    job, time_left = task_queue.dequeue()
    print('Running', job)
    sleep(1)
    time_left -= 1
    if time_left > 0:
        print('Re-queueuing', job, 'with remaining time =', time_left)
        task_queue.enqueue((job, time_left))
    else:
        print('*', job, 'done')

Running Job 0
Re-queueuing Job 0 with remaining time = 2
Running Job 1
Re-queueuing Job 1 with remaining time = 5
Running Job 2
Re-queueuing Job 2 with remaining time = 4
Running Job 0
Re-queueuing Job 0 with remaining time = 1
Running Job 1
Re-queueuing Job 1 with remaining time = 4
Running Job 2
Re-queueuing Job 2 with remaining time = 3
Running Job 0
* Job 0 done
Running Job 1
Re-queueuing Job 1 with remaining time = 3
Running Job 2
Re-queueuing Job 2 with remaining time = 2
Running Job 1
Re-queueuing Job 1 with remaining time = 2
Running Job 2
Re-queueuing Job 2 with remaining time = 1
Running Job 1
Re-queueuing Job 1 with remaining time = 1
Running Job 2
* Job 2 done
Running Job 1
* Job 1 done


### ... for doling out work

Queues are also frequently used as a sort of conveyer belt for multiple worker functions to draw from. Here we implement a "work queue" pattern used by multiple threads of execution:

In [84]:
from threading import Thread, Lock
from time import sleep
import random

lock = Lock()
def worker_fn(cid, q):
    while True:
        try:
            with lock:
                work = q.dequeue()
        except: # queue is empty
            sleep(1)
            continue
        if work == 'Stop':
            print('Consumer', cid, 'stopping.')
            break
        else:
            print('Consumer', cid, 'processing', work)
            sleep(random.random())

work_queue = Queue()
for i in range(5):
    Thread(target=worker_fn, args=(i, work_queue)).start()

In [85]:
import threading
threading.active_count()

10

In [86]:
for i in range(10):
    with lock:
        work_queue.enqueue(i)

In [87]:
for i in range(5):
    with lock:
        work_queue.enqueue('Stop')

## 3. Run-time analysis

Stack & Queue implementations:

- Insertion (push and enqueue) = $O(1)$
- Deletion (pop and dequeue) = $O(1)$